This notebook checks the combined April 9-11 ground station data by plotting all of the frames.. watch out! ;)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# --- Load the saved DataFrame from previous code ---
df = pd.read_parquet("combined_ionex_vtec.parquet")

# Ensure columns are MultiIndex [lat, lon]
if not isinstance(df.columns, pd.MultiIndex):
    raise RuntimeError("Expected MultiIndex columns for [lat, lon]!")

# Get unique, sorted lat/lon values (match DataFrame column order)
lats = sorted(set(lat for lat, _ in df.columns), reverse=True)  # usually descending
lons = sorted(set(lon for _, lon in df.columns))

# (Optional sanity check)
print(f"Lats: {lats[:3]} ... {lats[-3:]}")
print(f"Lons: {lons[:3]} ... {lons[-3:]}")
print(f"Data shape: {df.shape}")

# For each epoch, plot the VTEC grid
for epoch in df.index:
    data_1d = df.loc[epoch].values
    grid = data_1d.reshape((len(lats), len(lons)))
    
    plt.figure(figsize=(13, 6))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    im = ax.pcolormesh(lons, lats, grid, transform=ccrs.PlateCarree(), shading='auto')
    plt.colorbar(im, ax=ax, orientation='vertical', label='VTEC (TECU)')
    plt.title(f'Global VTEC Map\n{epoch}')
    plt.tight_layout()
    plt.show()
    # Or to save: plt.savefig(f'vtec_map_{epoch:%Y%m%d_%H%M%S}.png'); plt.close()


# The next section creates animation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.animation as animation

# --- Load Data ---
df = pd.read_parquet("combined_ionex_vtec.parquet")

# --- Extract lat/lon, ensure correct order ---
if not isinstance(df.columns, pd.MultiIndex):
    raise RuntimeError("Expected MultiIndex columns for [lat, lon]!")

lats = sorted(set(lat for lat, _ in df.columns), reverse=True)  # usually top-to-bottom
lons = sorted(set(lon for _, lon in df.columns))

# --- Precompute all grids for animation (faster) ---
all_grids = [df.loc[epoch].values.reshape((len(lats), len(lons))) for epoch in df.index]
vmin = np.nanmin([np.nanmin(g) for g in all_grids])
vmax = np.nanmax([np.nanmax(g) for g in all_grids])

# --- Plotting setup ---
fig = plt.figure(figsize=(13, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()

# Use meshgrid for pcolormesh edge coordinates
lon_edges = np.linspace(min(lons), max(lons), len(lons)+1)
lat_edges = np.linspace(min(lats), max(lats), len(lats)+1)

im = ax.pcolormesh(lon_edges, lat_edges, all_grids[0], 
                   transform=ccrs.PlateCarree(), shading='auto', vmin=vmin, vmax=vmax)
cb = plt.colorbar(im, ax=ax, orientation='vertical', label='VTEC (TECU)')
title = plt.title(f'Global VTEC Map\n{df.index[0]}')

def update(frame_idx):
    im.set_array(all_grids[frame_idx].flatten())
    title.set_text(f'Global VTEC Map\n{df.index[frame_idx]}')
    return im, title

ani = animation.FuncAnimation(
    fig, update, frames=len(df), interval=250, blit=False
)

# --- Save as GIF ---
ani.save('your_name_vtec_animationcombo.gif', writer='pillow', fps=4)

# --- For Jupyter notebook inline display ---
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000  # 50 MB (change as needed)
HTML(ani.to_jshtml())

# --- Save as MP4 (requires ffmpeg) ---
# ani.save('vtec_animation.mp4', writer='ffmpeg', fps=4)
